In [11]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print (tf.__version__)

2.18.0


loading dataset


In [12]:
from pandas.core import indexing
column_names = ['MPG', 'Cylinders', 'Displacement', 'Hoorsepower',
                'weight', 'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv('/content/auto-mpg.csv', names= column_names,
                          na_values="?", comment='\t',
                          sep=",",index_col=False, skiprows=1)

dataset = raw_dataset.copy()
dataset.tail()

/tmp/ipython-input-12-2722153569.py:5: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  raw_dataset = pd.read_csv('/content/auto-mpg.csv', names= column_names,


,MPG,Cylinders,Displacement,Hoorsepower,weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


cheching for missing values


In [13]:
dataset.isna().sum()

,0
MPG,0
Cylinders,0
Displacement,0
Hoorsepower,6
weight,0
Acceleration,0
Model Year,0
Origin,0


In [14]:
dataset = dataset.dropna()
dataset

,MPG,Cylinders,Displacement,Hoorsepower,weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


handling categoricovalues


In [15]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe']= (origin == 2) * 1.0
dataset['Japan'] = (origin == 3)* 1.0
dataset.tail()

,MPG,Cylinders,Displacement,Hoorsepower,weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720,19.4,82,1.0,0.0,0.0


training and testing dataset

In [16]:
train_dataset = dataset.sample(frac= 0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [17]:
train_stats = train_dataset.describe()
train_stats

,MPG,Cylinders,Displacement,Hoorsepower,weight,Acceleration,Model Year,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.310510,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,7.728652,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,10.000000,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,17.000000,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,22.000000,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,28.950000,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,46.600000,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


separating output from the input

In [18]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [19]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Hoorsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


model training

In [20]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def build_model():
  model = keras.Sequential([
      layers.Dense(64, activation= 'relu', input_shape=[len(train_dataset.keys())]),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
      ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [21]:
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,865 (19.00 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
EPOCHS = 1000
history = model.fit(
    normed_train_data, train_labels, epochs= EPOCHS
)

Epoch 1/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 592.3557 - mae: 23.1054 - mse: 592.3557  
Epoch 2/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 528.0555 - mae: 21.5939 - mse: 528.0555 
Epoch 3/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 456.3307 - mae: 20.0696 - mse: 456.3307 
Epoch 4/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 395.6208 - mae: 18.4695 - mse: 395.6208 
Epoch 5/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 305.1356 - mae: 16.0056 - mse: 305.1356 
Epoch 6/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 229.2027 - mae: 13.6079 - mse: 229.2027 
Epoch 7/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 154.0847 - mae: 10.7337 - mse: 154.0847 
Epoch 8/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 103.8247 - mae: 8.5746 - mse: 103.8247
Epoch 9/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 68.1281 - mae: 6.7637 - mse: 68.1281 
Epoch 10/1000
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 43.4427 - mae: 5.2859 - mse: 43.44

Model testing

In [23]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
loss,mae,mse

(5.124146938323975, 1.7581497430801392, 5.124146938323975)

In [24]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


array([13.591907 ,  9.276216 ,  9.836257 , 22.863722 , 19.845085 ,
       13.381748 , 13.683387 , 12.657746 , 20.451365 , 28.58726  ,
       22.873165 , 20.179663 , 15.359804 , 24.516224 , 12.767336 ,
       14.474951 , 14.757847 , 13.420555 , 16.71871  , 12.374827 ,
       14.268405 , 23.90452  , 19.438656 , 20.785204 , 29.628542 ,
       22.422586 , 15.886775 , 22.077705 , 18.209028 , 16.614862 ,
       28.416662 , 23.101164 , 18.00906  , 20.64351  , 26.822779 ,
       15.9441805, 18.256811 , 25.455875 , 28.882051 , 18.065409 ,
       31.087008 , 29.797138 , 14.298559 , 38.786896 , 36.440094 ,
       36.39653  , 19.086313 , 19.67573  , 18.337454 , 24.410013 ,
       31.962439 , 18.052797 , 32.737934 , 18.494234 , 17.231268 ,
       18.998331 , 34.618683 , 35.284607 , 19.564526 , 25.115232 ,
       42.81066  , 32.587635 , 26.917963 , 26.294456 , 31.388483 ,
       37.815186 , 35.556747 , 35.89048  , 37.22577  , 32.49681  ,
       21.227098 , 29.803019 , 28.842329 , 29.750738 , 38.8176

In [25]:
test_labels

,MPG
9,15.0
25,10.0
28,9.0
31,25.0
33,19.0
...,...
369,34.0
375,36.0
382,34.0
384,32.0


Model Conversion

In [27]:
kearas_file = "automobile.h5"
tf.keras.models.save_model(model, kearas_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("automobile.tflite", "wb").write(tfmodel)

Saved artifact at '/tmp/tmpzme8gwws'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 9), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134635099587984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134635099592400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134635099590672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134635099588368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134635099592976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134635099589136: TensorSpec(shape=(), dtype=tf.resource, name=None)


21344